In [1]:
# import packages
import geopandas as gpd
import osmnx as ox
import momepy
import pandana as pdna
import pandas as pd
import numpy as np
import networkx as nx

from scipy.spatial import cKDTree
from shapely.geometry import Point

from shapely.geometry import box

In [2]:
## load data

# load geopackages
lsoas = gpd.read_file(r'C:\Users\b8008458\Documents\2021_2022\Scratch Space\York\Centriods\YorkLSOAsSinglePartGPKG.gpkg')


# get location
place = "York, United Kingdom"
walk_dist = 1200 # distance to be walked
no_pois = 700 # max number of points to look for
type = 'walk'  # network type to get. walk, bike, all or drive

cities = ox.geocode_to_gdf([place])

# get all amenities for a given study area

tags = {"amenity": ["bar","cafe","pub","restaurant",
                    "college","kindergarten","library","school","university", "childcare",
                    ,"bicycle_parking","bicycle_repair_station","bicycle_rental","bus_station","ferry_terminal",
                    "taxi","atm","bank","bureau_de_change",
                    "clinic","dentist","doctors","hospital","pharmacy","social_facility","veterinary"
                    ,"arts_centre","cinema","community_centre","public_bookcase", "kitchen",
                    "social_centre", "theatre", "marketplace", "place_of_worship",
                    "police", "fire_station","post_box", "post_depot", "post_office", "townhall",
                    "drinking_water","toilets","water_point", "parcel_locker", "shower", "telephone",
                    "recycling"]}

pois = ox.geometries_from_place(place, tags)

# load streets from OSM
walk_graph = ox.graph_from_place(place, network_type = type) # download walking network

# get some barriers
barriers = gpd.read_file(r"C:\Users\b8008458\Documents\2021_2022\Scratch Space\York\Barriers\YorkBarriers.shp")


c:\Users\b8008458\Anaconda3\envs\ox\lib\site-packages\osmnx\geometries.py:805: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):
c:\Users\b8008458\Anaconda3\envs\ox\lib\site-packages\osmnx\geometries.py:805: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):


In [3]:
## clean data

# streets
walk_graph = ox.get_undirected(walk_graph) # cleans the network keeping parallel edges only if geometry is different
walk_graph = ox.projection.project_graph(walk_graph) # project graph
walk_streets = ox.graph_to_gdfs(walk_graph, nodes=False, edges=True, node_geometry=False, fill_edge_geometry=True) # convert graph to gdf
walk_streets = walk_streets[["geometry", "from", "to", "length"]] # clean columns
walk_streets = walk_streets.to_crs(27700) # set crs

In [4]:
# lsoas
lsoas = lsoas.reset_index(drop=True).explode().reset_index(drop=True) # clean
lsoas['lsoaID'] = range(0,len(lsoas)) # generate index column
lsoas = lsoas.set_index('lsoaID') # set index
lsoas = lsoas.to_crs(27700) # set crs

C:\Users\b8008458\AppData\Local\Temp\ipykernel_24216\2443919015.py:2: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  lsoas = lsoas.reset_index(drop=True).explode().reset_index(drop=True) # clean


In [5]:
# pois

pois = pois.reset_index(drop=True).explode().reset_index(drop=True) # avoid multipart pois
pois['poiID'] = range(0,len(pois))
pois = pois.set_index('poiID')

# convert polygons to points
pois['geometry'] = pois.centroid

# clean columns
pois = pois[['geometry','amenity']]
pois = pois.to_crs(27700) # set crs

C:\Users\b8008458\AppData\Local\Temp\ipykernel_24216\598720326.py:3: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  pois = pois.reset_index(drop=True).explode().reset_index(drop=True) # avoid multipart pois
C:\Users\b8008458\AppData\Local\Temp\ipykernel_24216\598720326.py:8: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  pois['geometry'] = pois.centroid


In [6]:
## generate pandana network
# nodes and edges for walk network
nodes_walk, edges_walk = momepy.nx_to_gdf( # convert network to gdf
    momepy.gdf_to_nx( # convert to nx graph
        walk_streets.explode() # remove multipart rows
    )
)
nodes_walk = nodes_walk.set_index('nodeID') # set index

# generate walk pandana network
walk_streets_pdna = pdna.Network( 
    nodes_walk.geometry.x,
    nodes_walk.geometry.y,
    edges_walk['node_start'], # set origins
    edges_walk['node_end'], # set destinations
    edges_walk[['mm_len']] # set edge length
)

C:\Users\b8008458\AppData\Local\Temp\ipykernel_24216\1581756382.py:5: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  walk_streets.explode() # remove multipart rows


In [7]:
# attach pois to the network
walk_streets_pdna.set_pois( # snap pois to network
    category = 'pois', # set name of the new layer snapped on the network
    maxdist = walk_dist, # set maximum distance
    maxitems = no_pois, # set maximum number of pois to look for
    x_col = pois.geometry.x,
    y_col = pois.geometry.y
)
results = walk_streets_pdna.nearest_pois( # calculate distances to pois
    distance = walk_dist, # maximum distance
    category = 'pois', # layer where we want to look for
    num_pois = no_pois, # max number of pois to look for
    include_poi_ids = True # include pois ids
)

c:\Users\b8008458\Anaconda3\envs\ox\lib\site-packages\pandana\network.py:660: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  elif isinstance(maxitems, type(pd.Series())):
c:\Users\b8008458\Anaconda3\envs\ox\lib\site-packages\pandana\network.py:668: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  elif isinstance(maxdist, type(pd.Series())):


In [8]:
# store results separately as distances and poiIDs

# separate distances from poi ids
distances = results.iloc[:,:round(len(results.columns)/2,)] # create df with distances
pois_ids = results.iloc[:,round(len(results.columns)/2,):] # create df with pois ids

# convert wide matrices to long
distances_long = pd.melt(distances.reset_index(), id_vars='nodeID',value_name='distance') # make matrix long
pois_ids_long = pd.melt(pois_ids.reset_index(), id_vars='nodeID',value_name='poiID') # make matrix long

# create an od long df containing nodeID, distance, and poiID
od = distances_long
od['poiID'] = pois_ids_long['poiID'].astype('Int64') # set a column with pois ids (as they are indexed, they are already in the right order)

# format od matrix and drop NAs
od = od[['nodeID','poiID','distance']] # clean columns
od = od.dropna() # drop NAs

In [9]:
# merge od data with POIs data
pois = pois.reset_index() # reset index pois df


In [10]:
od_pois_info = pd.merge(od, pois[['amenity','poiID']].reset_index(), left_on='poiID', right_on='poiID') # merge pois info to od matrix

In [11]:
od_pois_info

,nodeID,poiID,distance,index,amenity
0,1,770,617.469971,770,bicycle_parking
1,2,770,624.268982,770,bicycle_parking
2,3,770,699.742981,770,bicycle_parking
3,4,770,677.161011,770,bicycle_parking
4,6,770,590.724976,770,bicycle_parking
...,...,...,...,...,...
2380310,22312,1282,473.149994,1282,restaurant
2380311,22313,1282,403.635010,1282,restaurant
2380312,22315,1282,388.140015,1282,restaurant
2380313,134,1282,1167.176025,1282,restaurant


In [12]:
# add lsoa information to the od_pois_info 
lsoa_nodes = walk_streets_pdna.get_node_ids( # get nearest street nodes to each postcode
    lsoas.geometry.x,
    lsoas.geometry.y
)
lsoa_nodes = gpd.GeoDataFrame(lsoa_nodes).reset_index() # reset index
lsoa_nodes = lsoa_nodes.rename(columns={'node_id':'nodeID'}) # change col names



In [13]:
# get lsoa geometries
lsoa_nodes = lsoa_nodes.merge(lsoas, on='lsoaID', how='left', suffixes=('','_y'))
lsoa_nodes.drop(lsoa_nodes.filter(regex='_y$').columns, axis =1, inplace=True)
lsoa_nodes = lsoa_nodes[['lsoaID','nodeID','geometry']]

In [14]:
# add lsoas to od_pois_info
od_pois_info = pd.merge(od_pois_info, lsoa_nodes, left_on='nodeID', right_on='nodeID') # add lsoa to od matrix

# to csv file 
od_pois_info.to_csv(r'C:\Users\b8008458\Documents\2021_2022\Scratch Space\od_pois_info.csv', index=False)

In [15]:
# save to geospatail format
od_pois_info = gpd.GeoDataFrame(od_pois_info, geometry = 'geometry')
od_pois_info.to_file(r'C:\Users\b8008458\Documents\2021_2022\Scratch Space\od_pois_info.shp', index=False)

In [16]:
# count the number of aminites accessable from a given lsoa
lsoa_level_accessablity = od_pois_info.groupby("lsoaID").size()

In [17]:
# convert to dataframe
lsoa_level_accessablity = pd.DataFrame(lsoa_level_accessablity)

# merge with orignal lsoa file
lsoas = lsoas.merge(lsoa_level_accessablity, on='lsoaID', how='left', suffixes=('','_y'))
lsoas.drop(lsoas.filter(regex='_y$').columns, axis =1, inplace=True)
lsoas.rename(columns = {0:'amenity count'}, inplace = True)

In [18]:
# output to file
lsoas.to_file(r"C:\Users\b8008458\Documents\2021_2022\Scratch Space\YorkLSOASTEST.gpkg")

Now repeat process with a broken network

In [19]:
# create a network with edges with barriers on removed to simulate not being able to pass through a barrier
G = ox.graph_from_place(place, simplify=True, network_type=type)
G_edges = ox.graph_to_gdfs(ox.get_undirected(G), nodes=False, edges=True, node_geometry=False, fill_edge_geometry=True)
G_nodes = ox.graph_to_gdfs(ox.get_undirected(G), nodes=True, edges=False, node_geometry=True, fill_edge_geometry=False)
shply_line = G_edges.geometry.unary_union 
point = barriers.to_crs(G_edges.crs)
for i in range(len(point)):
    print(shply_line.interpolate(shply_line.project( point.geometry[i])).wkt) # snap points to line
result = point.copy()
result['geometry'] = result.apply(lambda row: shply_line.interpolate(shply_line.project( row.geometry)), axis=1)
buffer = result.geometry.buffer(0.00001) # create tiny buffer around points
broken_network = G_edges.intersects(buffer.unary_union)
broken_network = pd.DataFrame(broken_network)
broken_network.rename(columns = {0:'Clipped'}, inplace = True) # clip network
broken_network = pd.concat([broken_network,G_edges],axis=1)
# drop intersected lines
broken_network.drop(broken_network[broken_network['Clipped'] == True].index, inplace=True)
broken_network = broken_network[['geometry','to','from']]
broken_network = gpd.GeoDataFrame(broken_network, geometry='geometry')
G_edges = broken_network.to_crs(3857)
G_edges['length'] = G_edges.length
G_edges = G_edges.to_crs(4326)
G_edges = G_edges.to_crs(27700)
# G = ox.graph_from_gdfs(G_nodes, G_edges)

#G_after_processing = ox.graph_from_gdfs(G_nodes, G_edges)
#G_after_processing2 = momepy.nx_to_gdf(G_after_processing)
#print(G_after_processing)

POINT (-1.083041216634224 53.94557926654369)
POINT (-1.0739283 53.9441324)
POINT (-1.0739283 53.9441324)
POINT (-1.0739283 53.9441324)
POINT (-1.0797802 53.9450607)
POINT (-1.0738031 53.9451816)
POINT (-1.0732171 53.9470538)
POINT (-1.066155850653157 53.946130392666426)
POINT (-1.059554666188326 53.947470729338484)
POINT (-1.058435955680703 53.949930480875096)
POINT (-1.0534923043921502 53.94992674753619)
POINT (-1.048532460812328 53.94932347191377)
POINT (-1.0470913631995158 53.95011685417154)
POINT (-1.0449237655577528 53.950045630109955)
POINT (-1.045820272112785 53.95360492149316)
POINT (-1.0506803074754514 53.95973791439855)
POINT (-1.04955 53.96077)
POINT (-1.0458818 53.9593264)
POINT (-1.043527672911762 53.960495085014195)
POINT (-1.0383203937939562 53.96134269184238)
POINT (-1.057353891483021 53.95972082289049)
POINT (-1.0389224 54.0334809)
POINT (-1.060401967477918 53.9596528574473)
POINT (-1.0647184 53.9609276)
POINT (-1.0684667 53.9595748)
POINT (-1.073285871801579 53.958339

c:\Users\b8008458\Anaconda3\envs\ox\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
C:\Users\b8008458\AppData\Local\Temp\ipykernel_24216\645428067.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  buffer = result.geometry.buffer(0.00001) # create tiny buffer around points


In [20]:
## clean data

# streets
walk_streets = G_edges[["geometry", "from", "to", "length"]] # clean columns
walk_streets = walk_streets.to_crs(27700) # set crs

In [21]:
## generate pandana network
# nodes and edges for walk network
nodes_walk, edges_walk = momepy.nx_to_gdf( # convert network to gdf
    momepy.gdf_to_nx( # convert to nx graph
        walk_streets.explode() # remove multipart rows
    )
)
nodes_walk = nodes_walk.set_index('nodeID') # set index

# generate walk pandana network
walk_streets_pdna = pdna.Network( 
    nodes_walk.geometry.x,
    nodes_walk.geometry.y,
    edges_walk['node_start'], # set origins
    edges_walk['node_end'], # set destinations
    edges_walk[['mm_len']] # set edge length
)

C:\Users\b8008458\AppData\Local\Temp\ipykernel_24216\1581756382.py:5: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  walk_streets.explode() # remove multipart rows


In [22]:
# attach pois to the network
walk_streets_pdna.set_pois( # snap pois to network
    category = 'pois', # set name of the new layer snapped on the network
    maxdist = walk_dist, # set maximum distance
    maxitems = no_pois, # set maximum number of pois to look for
    x_col = pois.geometry.x,
    y_col = pois.geometry.y
)
results = walk_streets_pdna.nearest_pois( # calculate distances to pois
    distance = walk_dist, # maximum distance
    category = 'pois', # layer where we want to look for
    num_pois = no_pois, # max number of pois to look for
    include_poi_ids = True # include pois ids
)

c:\Users\b8008458\Anaconda3\envs\ox\lib\site-packages\pandana\network.py:660: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  elif isinstance(maxitems, type(pd.Series())):
c:\Users\b8008458\Anaconda3\envs\ox\lib\site-packages\pandana\network.py:668: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  elif isinstance(maxdist, type(pd.Series())):


In [23]:
# store results separately as distances and poiIDs

# separate distances from poi ids
distances = results.iloc[:,:round(len(results.columns)/2,)] # create df with distances
pois_ids = results.iloc[:,round(len(results.columns)/2,):] # create df with pois ids

# convert wide matrices to long
distances_long = pd.melt(distances.reset_index(), id_vars='nodeID',value_name='distance') # make matrix long
pois_ids_long = pd.melt(pois_ids.reset_index(), id_vars='nodeID',value_name='poiID') # make matrix long

# create an od long df containing nodeID, distance, and poiID
od = distances_long
od['poiID'] = pois_ids_long['poiID'].astype('Int64') # set a column with pois ids (as they are indexed, they are already in the right order)

# format od matrix and drop NAs
od = od[['nodeID','poiID','distance']] # clean columns
od = od.dropna() # drop NAs

In [24]:
# merge od data with POIs data
pois = pois.reset_index() # reset index pois df

In [25]:
od_pois_info = pd.merge(od, pois[['amenity','poiID']].reset_index(), left_on='poiID', right_on='poiID') # merge pois info to od matrix

In [26]:
# add lsoa information to the od_pois_info 
lsoa_nodes = walk_streets_pdna.get_node_ids( # get nearest street nodes to each postcode
    lsoas.geometry.x,
    lsoas.geometry.y
)
lsoa_nodes = gpd.GeoDataFrame(lsoa_nodes).reset_index() # reset index
lsoa_nodes = lsoa_nodes.rename(columns={'node_id':'nodeID'}) # change col names


In [27]:
# get lsoa geometries
lsoa_nodes = lsoa_nodes.merge(lsoas, on='lsoaID', how='left', suffixes=('','_y'))
lsoa_nodes.drop(lsoa_nodes.filter(regex='_y$').columns, axis =1, inplace=True)
lsoa_nodes = lsoa_nodes[['lsoaID','nodeID','geometry']]

In [28]:
# add lsoas to od_pois_info
od_pois_info = pd.merge(od_pois_info, lsoa_nodes, left_on='nodeID', right_on='nodeID') # add lsoa to od matrix

# to csv file 
od_pois_info.to_csv(r'C:\Users\b8008458\Documents\2021_2022\Scratch Space\od_pois_info_barrier.csv', index=False)

In [29]:
# save to geospatail format
od_pois_info = gpd.GeoDataFrame(od_pois_info, geometry = 'geometry')
od_pois_info.to_file(r'C:\Users\b8008458\Documents\2021_2022\Scratch Space\od_pois_info_barrier.shp', index=False)

In [30]:
# count the number of aminites accessable from a given lsoa
lsoa_level_accessablity = od_pois_info.groupby("lsoaID").size()

In [31]:
# convert to dataframe
lsoa_level_accessablity = pd.DataFrame(lsoa_level_accessablity)

# merge with orignal lsoa file
lsoas = lsoas.merge(lsoa_level_accessablity, on='lsoaID', how='left', suffixes=('','_y'))
lsoas.drop(lsoas.filter(regex='_y$').columns, axis =1, inplace=True)
lsoas.rename(columns = {0:'amenity count barriers'}, inplace = True)

In [33]:
lsoas['amenity_diff'] =  lsoas['amenity count'].sub(lsoas['amenity count barriers'], axis = 0)

In [34]:
# output to file
lsoas.to_file(r"C:\Users\b8008458\Documents\2021_2022\Scratch Space\YorkLSOASTESTBArrier.gpkg")

In [36]:
pois.to_file(r"C:\Users\b8008458\Documents\2021_2022\Scratch Space\pois.gpkg")